The dataset is from US Forest Service (USFS) Region2: Rocky Mountain region.

In [ ]:
!pip install tensorflow
!pip install keras_tuner

In [15]:
from pandas import read_csv

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

from keras_tuner.tuners import RandomSearch

from matplotlib import pyplot

'''from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

from tensorflow.keras.callbacks import EarlyStopping


import matplotlib.pyplot as plt
import numpy as np'''

'from tensorflow.keras.preprocessing.image import ImageDataGenerator\n\nfrom sklearn.metrics import classification_report\nfrom sklearn.metrics import confusion_matrix\n\nfrom tensorflow.keras.callbacks import EarlyStopping\n\n\nimport matplotlib.pyplot as plt\nimport numpy as np'

In [16]:
data = read_csv('cover_data.csv', header=0) #, parse_dates=[0], index_col=0, squeeze=True, date_parser=parser)
# summarize first few rows
print(data.shape)
print(data.columns)

(581012, 55)
Index(['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology',
       'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways',
       'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm',
       'Horizontal_Distance_To_Fire_Points', 'Wilderness_Area1',
       'Wilderness_Area2', 'Wilderness_Area3', 'Wilderness_Area4',
       'Soil_Type1', 'Soil_Type2', 'Soil_Type3', 'Soil_Type4', 'Soil_Type5',
       'Soil_Type6', 'Soil_Type7', 'Soil_Type8', 'Soil_Type9', 'Soil_Type10',
       'Soil_Type11', 'Soil_Type12', 'Soil_Type13', 'Soil_Type14',
       'Soil_Type15', 'Soil_Type16', 'Soil_Type17', 'Soil_Type18',
       'Soil_Type19', 'Soil_Type20', 'Soil_Type21', 'Soil_Type22',
       'Soil_Type23', 'Soil_Type24', 'Soil_Type25', 'Soil_Type26',
       'Soil_Type27', 'Soil_Type28', 'Soil_Type29', 'Soil_Type30',
       'Soil_Type31', 'Soil_Type32', 'Soil_Type33', 'Soil_Type34',
       'Soil_Type35', 'Soil_Type36', 'Soil_Type37', 'Soil_Type38',
       'Soil_Type39',

In [17]:
#check for any missing data
data.isnull().sum().sum()

0

In [18]:
data['class'].dropna(inplace=True) # drop any na values
print(data['class'].unique())
#print(data['Elevation'].max())
#print(data['Aspect'].min())
#print(data['Slope'].max())
#print(data['Horizontal_Distance_To_Hydrology'].min())
#print(data['Vertical_Distance_To_Hydrology'].min())
#print(data['Hillshade_9am'].max())
#print(data['Hillshade_Noon'].max())
#print(data['Hillshade_3pm'].max())
#print(data['Horizontal_Distance_To_Fire_Points'].max())
print(data['Wilderness_Area1'].unique())
print(data['Wilderness_Area2'].unique())
print(data['Wilderness_Area3'].unique())
print(data['Wilderness_Area4'].unique())

[5 2 1 7 3 6 4]
[1 0]
[0 1]
[0 1]
[0 1]


There are 55 columns in the dataset, and >500k records.
The label we will use for predictions is the 'class' column, which has 7 types, from 1-7.

The following columns of interest are:
* Elevation: in metres, with the max value of 3858
  * we'll use 4000 as the highest value when scaling between 0-1 for the NN. 
* Aspect: in degrees. Scale from 0-360
* Slope: In degrees. Maximum is 66. Scale from 0-70
* Horizontal distance to Hydrology. Distance to water source. Scale from 0 to 1400
* Vertical distance to Hydrology. Distance to water source. Scale from -180 to 650
* Hillshade at 9am. Scale from 0 - 254
* Hillshade at Noon. Scale from 0 - 254
* Hillshade at 3pm. Scale from 0 - 254
* Horizontal_Distance_To_Fire_Points. Scale from 0 - 7200
* Wilderness_Area 1-4: Binary columns
* Soil_Type 1-40: Binary Columns

We will convert the binary columns into a single integer column for the purposes of input into the NN model.

In [19]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 581012 entries, 0 to 581011
Data columns (total 55 columns):
 #   Column                              Non-Null Count   Dtype
---  ------                              --------------   -----
 0   Elevation                           581012 non-null  int64
 1   Aspect                              581012 non-null  int64
 2   Slope                               581012 non-null  int64
 3   Horizontal_Distance_To_Hydrology    581012 non-null  int64
 4   Vertical_Distance_To_Hydrology      581012 non-null  int64
 5   Horizontal_Distance_To_Roadways     581012 non-null  int64
 6   Hillshade_9am                       581012 non-null  int64
 7   Hillshade_Noon                      581012 non-null  int64
 8   Hillshade_3pm                       581012 non-null  int64
 9   Horizontal_Distance_To_Fire_Points  581012 non-null  int64
 10  Wilderness_Area1                    581012 non-null  int64
 11  Wilderness_Area2                    581012 non-null 

There are no blank columns so we should be ok to not have to remove unexpected data.

# Build the Model
Here we will use a sequential mode, with a Dense layer with 7 outputs to choose categorial labels.

In [20]:
# Reverse the one-hot encoded columns for the wilderness area columns
data['Wilderness_Area'] = data.loc[:, 'Wilderness_Area1':'Wilderness_Area4'].idxmax(axis=1)
print(data['Wilderness_Area'].unique())

['Wilderness_Area1' 'Wilderness_Area3' 'Wilderness_Area4'
 'Wilderness_Area2']


In [21]:
# Reverse the one-hot encoded columns for the Soil Type columns
data['Soil_Type'] = data.loc[:, 'Soil_Type1':'Soil_Type40'].idxmax(axis=1)
print(data['Soil_Type'].unique())

['Soil_Type29' 'Soil_Type12' 'Soil_Type30' 'Soil_Type18' 'Soil_Type16'
 'Soil_Type20' 'Soil_Type24' 'Soil_Type23' 'Soil_Type40' 'Soil_Type19'
 'Soil_Type8' 'Soil_Type22' 'Soil_Type39' 'Soil_Type9' 'Soil_Type38'
 'Soil_Type33' 'Soil_Type31' 'Soil_Type32' 'Soil_Type11' 'Soil_Type10'
 'Soil_Type5' 'Soil_Type28' 'Soil_Type4' 'Soil_Type1' 'Soil_Type13'
 'Soil_Type2' 'Soil_Type17' 'Soil_Type3' 'Soil_Type34' 'Soil_Type6'
 'Soil_Type14' 'Soil_Type37' 'Soil_Type35' 'Soil_Type36' 'Soil_Type21'
 'Soil_Type26' 'Soil_Type27' 'Soil_Type25' 'Soil_Type7' 'Soil_Type15']


In [22]:
data = data.drop(columns=[f"Wilderness_Area{i}" for i in range(1, 5)])
data = data.drop(columns=[f"Soil_Type{i}" for i in range(1, 41)])

In [23]:
# Change the datatypes of the new column to integer
data['Wilderness_Area'] = data['Wilderness_Area'].str.extract(r'(\d+)').astype(int)
data['Soil_Type'] = data['Soil_Type'].str.extract(r'(\d+)').astype(int)
print(data['Wilderness_Area'].dtypes)

int32


In [24]:
# The class column needs to be 0-6, not 1-7.
data['class'] = data['class'] - 1

## Train the model
First step is to split the data into train/test datasets

In [25]:
# split the data into features and labels
X = data.drop(columns=['class'])
y = data['class']

# normalise the feature set
scaler = StandardScaler()
X = scaler.fit_transform(X)

# split the features and labels into training,test, and validation sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=40)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=40)

In [26]:
model = keras.Sequential()
model.add(tf.keras.layers.Input(shape=(X_train.shape[1],))) # automatically select the number of inputs based on the shape of the data

model.add(tf.keras.layers.Dense(32, activation='relu'))

# output layer, 7+1 possible outcomes
model.add(tf.keras.layers.Dense(7, activation='softmax'))

learning_rate = 0.001
model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
                    loss='sparse_categorical_crossentropy',
                    metrics=['accuracy'])

model.fit(X_train, y_train, epochs=20, validation_data=(X_val, y_val))

Epoch 1/20
14526/14526 [==============================] - 27s 2ms/step - loss: 0.6491 - accuracy: 0.7278 - val_loss: 0.5986 - val_accuracy: 0.7472
Epoch 2/20
14526/14526 [==============================] - 25s 2ms/step - loss: 0.5918 - accuracy: 0.7493 - val_loss: 0.5798 - val_accuracy: 0.7549
Epoch 3/20
14526/14526 [==============================] - 26s 2ms/step - loss: 0.5748 - accuracy: 0.7569 - val_loss: 0.5655 - val_accuracy: 0.7588
Epoch 4/20
14526/14526 [==============================] - 28s 2ms/step - loss: 0.5645 - accuracy: 0.7606 - val_loss: 0.5591 - val_accuracy: 0.7615
Epoch 5/20
14526/14526 [==============================] - 26s 2ms/step - loss: 0.5586 - accuracy: 0.7632 - val_loss: 0.5575 - val_accuracy: 0.7655
Epoch 6/20
14526/14526 [==============================] - 28s 2ms/step - loss: 0.5544 - accuracy: 0.7654 - val_loss: 0.5510 - val_accuracy: 0.7669
Epoch 7/20
14526/14526 [==============================] - 28s 2ms/step - loss: 0.5514 - accuracy: 0.7667 - val_loss: 0

This initial mode is ok for a first attempt, at around 77% accuracy.

We need to optimise the model by adjusting the following parameters:
* batch size
* optimiser types
* learning rate
* hidden units
* number of layers
* implementation of dropout and early stopping
* activation functions

Using a Keras tuner can take care of a lot of the tuning for us!

In [28]:
# define a function that takes in the hyperparameter object, hp
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Input(shape=(X_train.shape[1],)))

    model.add(layers.Dense(hp.Choice('units', [32, 64, 128, 256]), activation='relu'))

    model.add(layers.Dense(7, activation='softmax'))

    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

    return model

tuner = RandomSearch(build_model, objective='val_accuracy', max_trials=5, overwrite=True, directory='my_tuner_directory')

tuner.search(X_train, y_train, epochs=20, validation_data=(X_val, y_val))
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

best_model = tuner.hypermodel.build(best_hps)
best_model.fit(X_train, y_train, epochs=20, validation_data=(X_val, y_val))

Trial 5 Complete [00h 09m 18s]
val_accuracy: 0.7912772297859192

Best val_accuracy So Far: 0.7983339428901672
Total elapsed time: 00h 46m 36s
Epoch 1/20
14526/14526 [==============================] - 27s 2ms/step - loss: 0.6244 - accuracy: 0.7385 - val_loss: 0.5753 - val_accuracy: 0.7552
Epoch 2/20
14526/14526 [==============================] - 26s 2ms/step - loss: 0.5623 - accuracy: 0.7606 - val_loss: 0.5469 - val_accuracy: 0.7677
Epoch 3/20
14526/14526 [==============================] - 26s 2ms/step - loss: 0.5434 - accuracy: 0.7684 - val_loss: 0.5387 - val_accuracy: 0.7711
Epoch 4/20
14526/14526 [==============================] - 26s 2ms/step - loss: 0.5319 - accuracy: 0.7737 - val_loss: 0.5235 - val_accuracy: 0.7780
Epoch 5/20
14526/14526 [==============================] - 26s 2ms/step - loss: 0.5230 - accuracy: 0.7780 - val_loss: 0.5146 - val_accuracy: 0.7828
Epoch 6/20
14526/14526 [==============================] - 26s 2ms/step - loss: 0.5154 - accuracy: 0.7819 - val_loss: 0.5106

The accuracy with this basic tuning didn't really improve things much; we're at 80% but can do better. Let's try adding some more layers and see how that improves things.

In [30]:
print(best_hps.values)

{'units': 64, 'learning_rate': 0.001}


In [31]:
# define a function that takes in the hyperparameter object, hp
def build_model2(hp):
    model = keras.Sequential()
    model.add(layers.Input(shape=(X_train.shape[1],)))

    units = hp.Int('units', min_value=32, max_value=512, step=32)
    layerNum = hp.Int('layers', min_value=2, max_value=7, step=1)
    for _ in range(layerNum):
        model.add(layers.Dense(units=units, activation='relu'))

    model.add(layers.Dense(7, activation='softmax'))

    hp_learning_rate = hp.Choice('learning_rate', values=[1e-4, 1e-5])

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

    return model

tuned2 = RandomSearch(build_model2, objective='val_accuracy', max_trials=5, overwrite=True, directory='my_tuner_directory')

tuned2.search(X_train, y_train, epochs=20, validation_data=(X_val, y_val))
best_hps = tuned2.get_best_hyperparameters(num_trials=1)[0]

best_model2 = tuned2.hypermodel.build(best_hps)
best_model2.fit(X_train, y_train, epochs=20, validation_data=(X_val, y_val))

Trial 5 Complete [00h 40m 42s]
val_accuracy: 0.9449235200881958

Best val_accuracy So Far: 0.9449235200881958
Total elapsed time: 02h 34m 59s
Epoch 1/20
14526/14526 [==============================] - 120s 8ms/step - loss: 0.5232 - accuracy: 0.7791 - val_loss: 0.4215 - val_accuracy: 0.8207
Epoch 2/20
14526/14526 [==============================] - 120s 8ms/step - loss: 0.3740 - accuracy: 0.8437 - val_loss: 0.3408 - val_accuracy: 0.8583
Epoch 3/20
14526/14526 [==============================] - 120s 8ms/step - loss: 0.3087 - accuracy: 0.8732 - val_loss: 0.2945 - val_accuracy: 0.8796
Epoch 4/20
14526/14526 [==============================] - 119s 8ms/step - loss: 0.2660 - accuracy: 0.8912 - val_loss: 0.2558 - val_accuracy: 0.8959
Epoch 5/20
14526/14526 [==============================] - 120s 8ms/step - loss: 0.2376 - accuracy: 0.9032 - val_loss: 0.2290 - val_accuracy: 0.9079
Epoch 6/20
14526/14526 [==============================] - 121s 8ms/step - loss: 0.2158 - accuracy: 0.9123 - val_loss: 

In [32]:
print(best_hps.values)

{'units': 352, 'layers': 6, 'learning_rate': 0.0001}


This improved things a lot, now with an accuracy of 94%!

## Test the model
We'll now evaluate the model using the test data that was set aside.


In [35]:
# Evaluate the model
test_loss, test_accuracy = best_model2.evaluate(X_test, y_test, verbose=2)
print(f'Test accuracy: {test_accuracy:.4f}')

1816/1816 - 6s - loss: 0.1351 - accuracy: 0.9464 - 6s/epoch - 3ms/step
Test accuracy: 0.9464


The accuracy seems to be similar to the rates previously shown. This could be misleading however, so let's see what the representation of the labels in the dataset is:

In [51]:
temp_data = data['class'] +1 
print(temp_data.value_counts())

2    283301
1    211840
3     35754
7     20510
6     17367
5      9493
4      2747
Name: class, dtype: int64


The data is heavily skewed towards classes of 2 and 1, so the model will be more accurate when fed corresponding information that relate to these classes, and less likely to predict classes of 4-6.

The training data could be normalised to present a more balanced training set so that the classes with fewer labels get a fair chance of being correctly labelled.

## Save the Model
This model can be saved as a `keras` file for deployment elsewhere, such as in a cloud service, or on an edge device.

In [54]:
best_model2.save("./coverage_model.keras")